## Package Imports

In [1]:
import os
from datetime import datetime

import librosa
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.layers import (Activation, Conv2D, Convolution2D, Dense, Dropout,
                          Flatten, GlobalAveragePooling2D, MaxPooling2D)
from keras.models import Sequential
from keras.optimizers import Adam
from keras.utils import np_utils
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Activation, Dense, Dropout, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tqdm import tqdm

## Loading of Dataset & Metadata

In [2]:
# Taga-kuha ng mga audio dataset at metadata
audio_dataset_path = 'C:/Users/Jerome/Desktop/Jerome/Programmierung Projekte/Thesis-Projekt/ACS-master-final/backend/data/dataset_train_clean/'
metadata = pd.read_csv('C:/Users/Jerome/Desktop/Jerome/Programmierung Projekte/Thesis-Projekt/ACS-master-final/backend/data/metadata_train_clean.csv')

## Feature Extraction

In [3]:
# Function para i-extract yung features ng audio
def features_extractor(file):
    audio, sample_rate = librosa.load(file_name, res_type = 'kaiser_fast') 
    mfccs_features = librosa.feature.mfcc(y = audio, sr = sample_rate, n_mfcc = 40)
    mfccs_scaled_features = np.mean(mfccs_features.T, axis = 0)
    
    return mfccs_scaled_features

In [4]:
# Iisa-isahin yung mga audio file sa folder at i-e-extract yung features gamit MFCC
extracted_features = []
for index_num,row in tqdm(metadata.iterrows()):
    file_name = os.path.join(os.path.abspath(audio_dataset_path), str(row["fileName"]))
    final_class_labels = row["className"]
    data = features_extractor(file_name)
    extracted_features.append([data, final_class_labels])
    
# Convert lahat ng features sa Panda dataframe
extracted_features_df = pd.DataFrame(extracted_features, columns = ['feature','class'])

775it [00:11, 66.48it/s]


In [5]:
# Taga-split ng dataset sa independent at dependent dataset
X = np.array(extracted_features_df['feature'].tolist())
y = np.array(extracted_features_df['class'].tolist())

# Taga-store ng classes
y = np.array(pd.get_dummies(y))

# Split yung train_test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

## CNN-RF Model

In [6]:
# Kung ilan yung classes
num_labels = y.shape[1]

cnnrf = Sequential()

# First layer ng model
cnnrf.add(Dense(100, input_shape = (40,)))
cnnrf.add(Activation('relu'))
cnnrf.add(Dropout(0.5))

# Second layer ng model
cnnrf.add(Dense(200))
cnnrf.add(Activation('relu'))
cnnrf.add(Dropout(0.5))

# Third layer ng model
cnnrf.add(Dense(100))
cnnrf.add(Activation('relu'))
cnnrf.add(Dropout(0.5))

# Fourth layer ng model
cnnrf.add(Dense(200))
cnnrf.add(Activation('relu'))
cnnrf.add(Dropout(0.5))

# Last layer ng model
cnnrf.add(Dense(num_labels))
cnnrf.add(Activation('softmax'))

cnnrf.compile(loss = 'categorical_crossentropy', metrics = ['accuracy'], optimizer = 'adam')

## CNN-RF Training

In [7]:
# Pag-train ng model
num_epochs = 110
num_batch_size = 64

checkpointer = ModelCheckpoint(filepath = 'model_checkpoints/cnnrf-v10.hdf5', 
                               verbose = 1, save_best_only = True)
start = datetime.now()

history = cnnrf.fit(X_train, y_train, batch_size = num_batch_size, epochs = num_epochs, validation_data = (X_test, y_test), callbacks = [checkpointer], verbose = 1)

duration = datetime.now() - start
print("Training duration ", duration)

Epoch 1/110
 1/10 [==>...........................] - ETA: 6s - loss: 50.8724 - accuracy: 0.2500
Epoch 1: val_loss improved from inf to 5.20774, saving model to model_checkpoints\cnnrf-v10.hdf5
10/10 [==============================] - 1s 37ms/step - loss: 31.9135 - accuracy: 0.2613 - val_loss: 5.2077 - val_accuracy: 0.3613
Epoch 2/110
 1/10 [==>...........................] - ETA: 0s - loss: 21.7003 - accuracy: 0.2500
Epoch 2: val_loss improved from 5.20774 to 2.14745, saving model to model_checkpoints\cnnrf-v10.hdf5
10/10 [==============================] - 0s 12ms/step - loss: 20.8469 - accuracy: 0.2565 - val_loss: 2.1475 - val_accuracy: 0.3742
Epoch 3/110
 1/10 [==>...........................] - ETA: 0s - loss: 17.3090 - accuracy: 0.3438
Epoch 3: val_loss improved from 2.14745 to 1.42278, saving model to model_checkpoints\cnnrf-v10.hdf5
10/10 [==============================] - 0s 14ms/step - loss: 13.8648 - accuracy: 0.2742 - val_loss: 1.4228 - val_accuracy: 0.4065
Epoch 4/110
 1/10 [=

In [8]:
test_accuracy = cnnrf.evaluate(X_train, y_train, verbose = 0)
print("Primary Training Accuracy: ", test_accuracy[1] * 100, "%")

test_accuracy = cnnrf.evaluate(X_test, y_test, verbose = 0)
print("Primary Testing Accuracy: ", test_accuracy[1] * 100, "%")

Primary Training Accuracy:  78.22580933570862 %
Primary Testing Accuracy:  80.0000011920929 %


## Anvil Server Connection

In [9]:
import anvil.server

#Anvil Direct Link - https://p47ett2nfjlaqqih.anvil.app/3ZRA6CFT35X2P4QOFJI3DGAV
anvil.server.connect("JOTCSN4YS2NNQR3ZBWJT2GBU-P47ETT2NFJLAQQIH")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default environment" as SERVER


In [10]:
import anvil.media
import anvil.mpl_util
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns

from glob import glob

import librosa
import librosa.display
import IPython.display as ipd

from itertools import cycle

@anvil.server.callable
def classify_audio(file):
    prediction_feature = features_extractor(file)
    prediction_feature = prediction_feature.reshape(1, -1)
    return cnnrf.predict(prediction_feature)

1/1 [==============================] - 0s 23ms/step
